In [1]:
import sys
print(sys.path)
sys.path.append('/Users/bytedance/PycharmProjects/github/CausalMatch')

import causalmatch as causalmatch
from causalmatch import matching,gen_test_data

print('current version is: ',causalmatch.__version__)

import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,roc_auc_score,f1_score
import statsmodels.api as sm

['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/bytedance/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages']
current version is:  0.0.2
/usr/local/bin/python3.12
3.12.2 (v3.12.2:6abddd9f6a, Feb  6 2024, 17:02:06) [Clang 13.0.0 (clang-1300.0.29.30)]
sys.version_info(major=3, minor=12, micro=2, releaselevel='final', serial=0)


# 20241028

In [2]:
df, rand_continuous, rand_true_param, param_te , rand_treatment, rand_error = gen_test_data(n = 10000, c_ratio=0.5)
X = ['c_1', 'c_2', 'c_3', 'd_1', 'gender']
y = ['y', 'y2']
id = 'user_id'

# treatment variable has to be a 0/1 dummy variable
# if is string, please convert to a 0/1 int input
T = 'treatment'

In [3]:
# -------------------------------------------------------- #
# STEP 1: initialize matching object
match_obj = matching(data = df,
                     T = T,
                     y = ['y'],
                     X = X,
                     id = id)
# -------------------------------------------------------- #

# STEP 2: propensity score matching
# match_obj.psm(n_neighbors = 1,  # number of neighbors
#               model = GradientBoostingClassifier(), # p-score model
#               trim_percentage = 0.1, # trim X% of pscore, if equals 0.1 then trim min 5% and max 5%
#               caliper = 0.1,
#               verbose = True) # p-score diff must be smaller than or equal to the caliper value

# if you with to keep all matched observations
match_obj.psm(n_neighbors = 1, 
              model = GradientBoostingClassifier(), 
              trim_percentage = 0, 
              caliper = 1) 

gamma = [1,1.5,2,2.5,3]
match_obj.robust_check(gamma)


             Robustness Check Output Table for Dep. Variable y              
                                                            coef    P>|t|   
  Average Treatment Effect:                                 0.47     0.00%  
                                                                            
  1. Total % of obs remained:                            101.94%      Pass  
   -- Treated % obs remained:                            100.00%      Pass  
   -- Control % obs remained:                            103.96%      Pass  
      -- The most repeated times of a control obs:            84         -  
  2. Sensitivity test result, Gamma statistics               1.5      Pass  
  3. Placebo Test Result conf. interval              -1.48,2.14,      Fail  


In [2]:
df, rand_continuous, rand_true_param, param_te , rand_treatment, rand_error = gen_test_data(n = 10000, c_ratio=0.5)
df.head()

X_1 = ['c_1']
X_2 = ['d_1']
X_3 = ['d_1', 'd_3']
X_4 = ['c_1', 'c_2', 'c_3']
X_5 = ['c_1', 'd_1', 'd_3']

y  = ['y']
id = 'user_id'
T  = 'treatment'
X_all = [X_1, X_2, X_3, X_4, X_5]

for X_i in X_all:
    match_obj = matching(data = df, T = T, X = X_i, id = id, y = y)

    match_obj.psm(n_neighbors = 1, model = LogisticRegression(), trim_percentage = 0.1, caliper = 0.1)
    match_obj.ate()

    res_post, res_pre = match_obj.balance_check(include_discrete = True)

    test_1 = [res_post['Mean Treated post-match'].sum(),res_post['Mean Control post-match'].sum()]
    test_2 = [res_pre['Mean Treated pre-match'].sum(), res_pre['Mean Control pre-match'].sum()]
    test_3 = match_obj.ate()
    test_3a = np.round(test_3['ate'].values[0],2)
        
    print(test_3a)

0.5
-0.78
0.45
0.44
0.47


In [20]:
match_obj = matching(data = df, T = T, X = X_1, id = id, y = y)
match_obj.psm(n_neighbors = 1, model = LogisticRegression(), trim_percentage = 0, caliper = 1)
match_obj.ate()

,y,ate,p_val
0,y,0.513355,8.394101e-29


In [21]:
X_balance_check, df_post_validate, df_pre_validate = data_process_bc(match_obj, True)

T      = match_obj.T
method = match_obj.method
y      = match_obj.y

df_post_validate_y, weight = data_process_ate(match_obj, df_post_validate)
dict_ate = {"y": [], "ate": [], "p_val": []}

x = df_post_validate_y[T]
x = sm.add_constant(x)

y_i = y[0]
Y = df_post_validate_y[y_i].values

model = sm.OLS(Y, x)
results = model.fit()

dict_ate['y'].append(y_i)
dict_ate['ate'].append(results.params[T])
dict_ate['p_val'].append(results.pvalues[T])
    
df_param = pd.DataFrame(dict_ate)

In [22]:
df_post_validate[df_post_validate['treatment']==0]['user_id'].value_counts()

8652    12
7591    11
8988     9
890      8
2635     8
        ..
6993     1
5629     1
7882     1
9309     1
7391     1
Name: user_id, Length: 2756, dtype: int64

In [24]:

df[X_3].dtypes




d_1    object
d_3    object
dtype: object

In [25]:
rand_continuous

array([[3.73012225e-01, 4.47996825e-01, 1.29440680e-01],
       [8.59878707e-01, 8.20388363e-01, 3.52053539e-01],
       [2.28887304e-01, 7.76783751e-01, 5.94783589e-01],
       ...,
       [1.17296364e-01, 4.51568702e-01, 2.15496503e-04],
       [4.89518874e-01, 9.49239564e-01, 2.73256016e-01],
       [8.15383174e-01, 5.28709272e-01, 6.10985008e-01]])

In [58]:
x

,const,treatment
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
...,...,...
10189,1.0,0.0
10190,1.0,0.0
10191,1.0,0.0
10192,1.0,0.0


[2.2399, 2.2389]

In [20]:
test_1 = [res_post.loc[0,'Mean Treated post-match'],res_post.loc[0,'Mean Control post-match']]
test_1

[0.4951, 0.4964]

In [22]:
np.testing.assert_almost_equal(test_1, [2.2399, 2.2389])

In [22]:

import numpy as np
import unittest
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from econml.dml import LinearDML
from econml.inference import BootstrapInference


class TestATE(unittest.TestCase):

    @classmethod
    def test_ate_inference(self):
        """Test the ate inference results."""
        df, rand_continuous, rand_true_param, param_te , rand_treatment, rand_error = gen_test_data(n = 10000, c_ratio=0.5)

        return 

        

AttributeError: 'tuple' object has no attribute 'head'

In [25]:
TestATE.setUpClass()
TestATE.test_ate_inference()

(array([[-1.51570586,  0.22780856,  0.2809105 ],
        [ 1.17477854,  2.48608502, -0.16033881],
        [ 1.06269697, -0.5773168 ,  1.10592052],
        ...,
        [ 1.11538882,  0.69876546, -0.38595966],
        [-1.34520417,  0.09307752,  1.86630958],
        [ 1.08859436,  1.00836284,  0.78456899]]),
 array([[1, 1],
        [0, 1],
        [0, 1],
        ...,
        [1, 1],
        [0, 0],
        [1, 0]]),
 array([[0.69646919, 0.28613933, 0.22685145],
        [0.55131477, 0.71946897, 0.42310646],
        [0.9807642 , 0.68482974, 0.4809319 ],
        ...,
        [0.50669599, 0.37539443, 0.50405586],
        [0.26927543, 0.61011736, 0.6219864 ],
        [0.20891734, 0.56747224, 0.4452157 ]]))

In [16]:
TestATEInference()

<__main__.TestATEInference testMethod=runTest>

In [6]:
match_obj = matching(data = df, 
                     y = y,
                     T = T, 
                     X = X,
                     id = 'user_id')

match_obj.cem(n_bins = 10, # number of bins you set to divide continuous variables, user pd.qcut function to obatin
              k2k = True)  # k2k: make sure number of treatment equals number of control. if is false, you need to apply weighted least square to obtain ATE




number of matched obs (9726, 15) number of total obs  (10000, 15)


In [7]:

T = match_obj.T
y = match_obj.y[0]


# 1. post-process data
X_balance_check, df_post_validate, df_pre_validate = data_process_bc(match_obj, True)
df_post_validate_y, weight = data_process_ate(match_obj, df_post_validate)

# 2. Single learner
X_mat = df_post_validate[X_balance_check].values
T_mat = df_post_validate[[T]].values
y_mat = df_post_validate_y[[y]].values

est_linear = s_learner_linear()
est_linear.fit(y_mat, T_mat, X_mat)
hte_linear = est_linear.predict(X_mat)


Cross Validation Scores:  [0.15 0.15 0.14 0.14 0.15]
Average CV Score:  0.14600000000000002
Number of CV Scores used in Average:  5


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, T_train, T_test, y_train, y_test = train_test_split(X_mat,T_mat, y_mat, test_size=0.33, random_state=42)


# nobs = T_mat[:, 0].shape[0]
# intercept = np.ones([nobs, 1])
# X_mat_full = np.concatenate((intercept,X_mat, T_mat), axis=1)
X_mat_full = np.concatenate((X_train, T_train), axis=1)
X_mat_full_test = np.concatenate((X_test, T_test), axis=1)
reg = LinearRegression().fit(X_mat_full, y_train)
reg.score(X_mat_full, y_train)


0.8120201901465802

In [9]:
y_pred = reg.predict(X_mat_full_test)
y_true = y_test
np.mean(np.abs((y_true - y_pred) / y_true))

0.17297136829003257

In [10]:
y_true

array([[6.84525613],
       [0.61228625],
       [9.40534546],
       ...,
       [6.01529475],
       [7.6671504 ],
       [4.38335752]])

In [11]:
y_pred

array([[5.8059716 ],
       [2.66184887],
       [7.05536864],
       ...,
       [5.45975828],
       [8.62697188],
       [3.65804321]])

In [6]:
nmape_scorer = make_scorer(NMAPE)
k_folds = KFold(n_splits = 5, shuffle=True, random_state=111)
scores = cross_val_score(LinearRegression(), 
                         X_mat_full, y_mat, 
                         scoring=nmape_scorer, cv=k_folds)


In [7]:
scores

array([3.12244908, 3.33660604, 3.25982898, 3.04192198, 4.32802582])

In [5]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

X = np.ones((3,2))
pf = PolynomialFeatures(3)
pf.fit(X)
print(pf.get_feature_names())

array([[0.37301223, 0.44799682, 0.12944068],
       [0.85987871, 0.82038836, 0.35205354],
       [0.13755356, 0.85289978, 0.23550748],
       ...,
       [0.61095956, 0.80160774, 0.89683825],
       [0.24087144, 0.38318785, 0.28364651],
       [0.48975841, 0.23475667, 0.76238795]])

In [6]:
y_mat

array([[-0.43404596],
       [-0.89965393],
       [ 0.91695661],
       ...,
       [-1.38210833],
       [ 0.34697248],
       [-1.33728904]])

In [7]:
T_mat

array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])

In [4]:
df = gen_test_data(n = 10000, c_ratio=0.5)
df.head()


X = ['c_1', 'c_2', 'c_3', 'd_1', 'gender']
y = ['y', 'y2']
id = 'user_id'

# treatment variable has to be a 0/1 dummy variable
# if is string, please convert to a 0/1 int input
T = 'treatment'

In [5]:
# STEP 1: initialize matching object
match_obj = matching(data = df, 
                       y = ['y'],
                       T = 'treatment', 
                       X = ['c_1','d_1','d_3'],
                       id = 'user_id')

# STEP 2: coarsened exact matching
match_obj.cem(n_bins = 10, # number of bins you set to divide continuous variables, user pd.qcut function to obatin
              k2k = True)  # k2k: make sure number of treatment equals number of control. if is false, you need to apply weighted least square to obtain ATE


# STEP 3: balance check after propensity score matching
res_post, res_pre = match_obj.balance_check(include_discrete=True)
print(res_post)

# STEP 4: obtain average partial effect 
match_obj.ate(use_weight = True)




number of matched obs (9768, 15) number of total obs  (10000, 15)
   Covariates  Mean Treated post-match  Mean Control post-match   SMD  \
0         c_1                   0.4999                   0.4983  0.01   
1     d_1_bee                   0.1114                   0.1194 -0.03   
2     d_1_cat                   0.1546                   0.1570 -0.01   
3     d_1_dog                   0.1929                   0.2002 -0.02   
4    d_1_pear                   0.2588                   0.2545  0.01   
5     d_3_1.0                   0.2039                   0.1882  0.04   
6     d_3_2.0                   0.0516                   0.0516  0.00   
7     d_3_3.0                   0.1790                   0.1804 -0.00   
8     d_3_4.0                   0.0704                   0.0713 -0.00   
9     d_3_5.0                   0.0692                   0.0676  0.01   
10    d_3_6.0                   0.0921                   0.0938 -0.01   
11    d_3_7.0                   0.1824                   0

,y,ate,p_val
0,y,0.512274,7.780753e-120


In [6]:
match_obj.ate(use_weight = True)

,y,ate,p_val
0,y,0.517945,1.781174e-122


In [30]:
# from causalmatch.matching.match_core.utils import data_process_bc, balance_check_x

# y      = match_obj.y
# id     = match_obj.id
# T      = match_obj.T
# method = match_obj.method
# df_out_ = match_obj.df_out_final

# X_balance_check, df_post_validate, df_pre_validate = data_process_bc(match_obj, True)

# if method == 'cem':
#     # 1. merge y variables from raw df to matched df
#     df_post_validate_y_ = df_post_validate.merge(match_obj.data[y + [id]], how='left', on=id)

#     # 2. for CEM need one more step due to WLS: merge weight to outcome dataframe
#     df_post_validate_y = df_post_validate_y_.merge(match_obj.df_out_final[[id,'weight_adjust']], how='left', on=id)
# else:
#     df_post_validate_y = df_post_validate.merge(match_obj.data[y + [id]], how='left', on=id)

# dict_ate = {"y" : [], "ate" : [], "p_val" : []}



In [31]:
# # Loop all y variables to do least square
# use_weight = True

# y_i = y[0]
# Y = df_post_validate_y[y_i]
# x = df_post_validate_y[T]
# weight = df_post_validate_y['weight_adjust']

# x = sm.add_constant(x)

# if method=='cem' and use_weight is True:

#     model = sm.WLS(Y, x, weights = weight)

# else:
#     model = sm.OLS(Y, x)
#     results = model.fit()



# results = model.fit()

In [32]:
# results.params

NameError: name 'use_weight' is not defined

In [14]:
df_post_validate_y[y_i]

NameError: name 'y_i' is not defined

In [16]:
# STEP 1: initialize matching object
match_obj = matching(data = df,     
                     T = T,
                     X = X,
                     id = id)

# STEP 2: propensity score matching

# match_obj.psm(n_neighbors = 1,  # number of neighbors
#               model = GradientBoostingClassifier(), # p-score model
#               trim_percentage = 0.1, # trim X% of pscore, if equals 0.1 then trim min 5% and max 5%
#               caliper = 0.1) # p-score diff must be smaller than or equal to the caliper value

match_obj.psm(n_neighbors = 1,  # number of neighbors
              model = GradientBoostingClassifier(), # p-score model
              trim_percentage = 0, # trim X% of pscore, if equals 0.1 then trim min 5% and max 5%
              caliper = 1) # p-score diff must be smaller than or equal to the caliper value


# STEP 3: balance check after propensity score matching
res_post, res_pre = match_obj.balance_check(include_discrete = True)




In [17]:
res_post

,Covariates,Mean Treated post-match,Mean Control post-match,SMD,Var Ratio,ks-p_val,ttest-p_val
0,c_1,0.5004,0.5145,-0.05,1.04,0.001,0.013
1,c_2,0.5024,0.4919,0.04,1.09,0.000,0.059
2,c_3,0.5033,0.4968,0.02,1.07,0.000,0.247
3,d_1_bee,0.1124,0.1112,0.00,NaN,1.000,0.850
4,d_1_cat,0.1581,0.1383,0.05,NaN,0.266,0.005
5,d_1_dog,0.1929,0.1787,0.04,NaN,0.684,0.067
6,d_1_pear,0.2551,0.2947,-0.09,NaN,0.001,0.000
7,gender_cat1,0.0714,0.0708,0.00,NaN,1.000,0.908
8,gender_cat2,0.0883,0.0820,0.02,NaN,1.000,0.256
9,gender_cat3,0.1683,0.1893,-0.06,NaN,0.209,0.006


In [18]:
res_pre

,Covariates,Mean Treated pre-match,Mean Control pre-match,SMD,Var Ratio,ks-p_val,ttest-p_val
0,c_1,0.5004,0.4984,0.01,0.99,0.406,0.722
1,c_2,0.5024,0.5062,-0.01,0.99,0.736,0.516
2,c_3,0.5033,0.5053,-0.01,0.99,0.904,0.736
3,d_1_bee,0.1124,0.1201,-0.02,NaN,0.998,0.229
4,d_1_cat,0.1581,0.1577,0.00,NaN,1.000,0.948
5,d_1_dog,0.1929,0.1997,-0.02,NaN,1.000,0.391
6,d_1_pear,0.2551,0.2541,0.00,NaN,1.000,0.916
7,gender_cat1,0.0714,0.0671,0.02,NaN,1.000,0.396
8,gender_cat2,0.0883,0.0938,-0.02,NaN,1.000,0.336
9,gender_cat3,0.1683,0.1691,-0.00,NaN,1.000,0.921


In [21]:
df_out = match_obj.df_out_final_post_trim.merge(df[y + X + [id]], how='left', on = id)
df_out[df_out['treatment']==0]['c_1'].mean()

0.5144666970681424

In [20]:
match_obj.df_out_final

,user_id_treat,treatment_treat,pscore_treat,user_id_control,treatment_control,pscore_control,pscore_diff
0,2,1.0,0.506268,2834,0.0,0.506247,0.000021
1,4,1.0,0.505473,320,0.0,0.505473,0.000000
2,5,1.0,0.507653,8968,0.0,0.507672,0.000019
3,6,1.0,0.452660,6961,0.0,0.452780,0.000120
4,7,1.0,0.525007,3201,0.0,0.525012,0.000006
...,...,...,...,...,...,...,...
5092,9994,1.0,0.508411,6397,0.0,0.508411,0.000000
5093,9995,1.0,0.528377,148,0.0,0.528399,0.000022
5094,9997,1.0,0.722550,5270,0.0,0.646283,0.076267
5095,9998,1.0,0.522738,3270,0.0,0.522727,0.000011


In [6]:
x_numeric = match_obj.X_numeric
x_discrete = match_obj.X_discrete
df_out_ = match_obj.df_out_final
df_cat = match_obj.data_with_categ
df_psm_post_trim = match_obj.df_out_final_post_trim
df_raw = match_obj.data

T = match_obj.T
id = match_obj.id

In [8]:
df_raw

,c_1,c_2,c_3,d_1,gender,d_3,treatment,y,y2,user_id
0,0.373012,0.447997,0.129441,cat,female,9.0,0.0,-0.434046,0.310434,0
1,0.859879,0.820388,0.352054,apple,cat5,7.0,0.0,-0.899654,0.703410,1
2,0.228887,0.776784,0.594784,dog,female,7.0,1.0,-2.425582,-1.560741,2
3,0.137554,0.852900,0.235507,bee,dog,3.0,0.0,0.916957,1.862262,3
4,0.146227,0.589869,0.574012,apple,dog,5.0,1.0,-0.199377,0.441810,4
...,...,...,...,...,...,...,...,...,...,...
9995,0.152349,0.201680,0.027182,pear,dog,2.0,1.0,-0.997052,-1.100664,9995
9996,0.258117,0.052080,0.832009,pear,female,6.0,0.0,-0.722280,0.225611,9996
9997,0.117296,0.451569,0.000215,dog,dog,9.0,1.0,2.560772,2.592424,9997
9998,0.489519,0.949240,0.273256,pear,cat2,3.0,1.0,-0.514552,0.418862,9998


In [10]:
include_discrete = True
if include_discrete:
    df_right = (df_cat) * 1
else:
    df_right = df_cat[x_numeric]

df_x = df_right.columns
df_right[id] = df_raw[id]
df_post_validate = df_psm_post_trim.merge(df_right, how='left', on=id)

In [11]:
df_right

,c_1,c_2,c_3,d_1_bee,d_1_cat,d_1_dog,d_1_pear,gender_cat1,gender_cat2,gender_cat3,gender_cat4,gender_cat5,gender_dog,gender_female,gender_male,gender_pig,user_id
0,0.373012,0.447997,0.129441,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,0.859879,0.820388,0.352054,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.228887,0.776784,0.594784,0,0,1,0,0,0,0,0,0,0,1,0,0,2
3,0.137554,0.852900,0.235507,1,0,0,0,0,0,0,0,0,1,0,0,0,3
4,0.146227,0.589869,0.574012,0,0,0,0,0,0,0,0,0,1,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.152349,0.201680,0.027182,0,0,0,1,0,0,0,0,0,1,0,0,0,9995
9996,0.258117,0.052080,0.832009,0,0,0,1,0,0,0,0,0,0,1,0,0,9996
9997,0.117296,0.451569,0.000215,0,0,1,0,0,0,0,0,0,1,0,0,0,9997
9998,0.489519,0.949240,0.273256,0,0,0,1,0,1,0,0,0,0,0,0,0,9998


In [12]:
df_x

Index(['c_1', 'c_2', 'c_3', 'd_1_bee', 'd_1_cat', 'd_1_dog', 'd_1_pear',
       'gender_cat1', 'gender_cat2', 'gender_cat3', 'gender_cat4',
       'gender_cat5', 'gender_dog', 'gender_female', 'gender_male',
       'gender_pig'],
      dtype='object')

In [13]:
df_post_validate

,user_id,treatment,pscore,c_1,c_2,c_3,d_1_bee,d_1_cat,d_1_dog,d_1_pear,gender_cat1,gender_cat2,gender_cat3,gender_cat4,gender_cat5,gender_dog,gender_female,gender_male,gender_pig
0,2,1.0,0.506268,0.228887,0.776784,0.594784,0,0,1,0,0,0,0,0,0,0,1,0,0
1,4,1.0,0.505473,0.146227,0.589869,0.574012,0,0,0,0,0,0,0,0,0,1,0,0,0
2,5,1.0,0.507653,0.061270,0.590426,0.245350,0,0,0,0,0,1,0,0,0,0,0,0,0
3,7,1.0,0.525007,0.037772,0.861549,0.753569,0,1,0,0,0,0,0,0,0,0,1,0,0
4,8,1.0,0.539596,0.405179,0.343526,0.170917,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9150,4820,0.0,0.510351,0.541758,0.052840,0.988058,0,0,1,0,0,0,0,0,1,0,0,0,0
9151,6397,0.0,0.508411,0.302329,0.513633,0.527378,0,1,0,0,0,0,0,0,0,0,1,0,0
9152,148,0.0,0.528399,0.070273,0.915035,0.215814,0,0,1,0,0,0,0,0,0,0,1,0,0
9153,3270,0.0,0.522727,0.189208,0.469702,0.359837,0,0,0,1,0,0,0,0,0,0,0,0,0


In [6]:
# STEP 4: get result - pandas df, and merge X and y back to original data
print('Output dataframe columns', match_obj.df_out_final_post_trim.columns)
df_out = match_obj.df_out_final_post_trim.merge(df[y + X + [id]], how='left', on = id)

# STEP 5: calculate average treatment effect 

X_mat = df_out[T]
y_mat = df_out[y]

X_mat = sm.add_constant(X_mat)
model = sm.OLS(y_mat,X_mat)
results = model.fit()
print(results.params)

Output dataframe columns Index(['user_id', 'treatment', 'pscore'], dtype='object')
                  0         1
const     -1.171885  0.034522
treatment  0.480457  0.085681


# 20240929

In [1]:
import sys
print(sys.path)
sys.path.append('/Users/bytedance/PycharmProjects/github/CausalMatch')

import causalmatch as causalmatch
from causalmatch import matching,gen_test_data

print('current version is: ',causalmatch.__version__)

import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)


import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,roc_auc_score,f1_score
import statsmodels.api as sm
from scipy.stats import wilcoxon,norm

['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/bytedance/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages']
current version is:  0.0.2
/usr/local/bin/python3.12
3.12.2 (v3.12.2:6abddd9f6a, Feb  6 2024, 17:02:06) [Clang 13.0.0 (clang-1300.0.29.30)]
sys.version_info(major=3, minor=12, micro=2, releaselevel='final', serial=0)


In [2]:
df, rand_continuous, rand_true_param, param_te , rand_treatment, rand_error = gen_test_data(n = 10000, c_ratio=0.5)
X = ['c_1', 'c_2', 'c_3', 'd_1', 'gender']
y = ['y', 'y2']
id = 'user_id'

# treatment variable has to be a 0/1 dummy variable
# if is string, please convert to a 0/1 int input
T = 'treatment'

In [3]:
# STEP 1: initialize matching object
match_obj = matching(data = df,
                     T = T,
                     X = X,
                     y = y,
                     id = id)

# STEP 2: propensity score matching
match_obj.psm(n_neighbors = 1,  # number of neighbors
              model = GradientBoostingClassifier(), # p-score model
              trim_percentage = 0.1, # trim X% of pscore, if equals 0.1 then trim min 5% and max 5%
              caliper = 0.1,
              verbose = True) # p-score diff must be smaller than or equal to the caliper value

# STEP 3: balance check after propensity score matching
res_post, res_pre = match_obj.balance_check(include_discrete = True)
print('post-matching balance check reseult',res_post) # pre-match balance check
print('pre-matching balance check reseult',res_pre)  # post-match balance check

# STEP 4: get result - pandas df, and merge X and y back to original data
print('Output dataframe columns', match_obj.df_out_final_post_trim.columns)
df_out = match_obj.df_out_final_post_trim.merge(df[y + X + [id]], how='left', on = id)

# STEP 5: calculate average treatment effect 

X_mat = df_out[T]
y_mat = df_out[y]

X_mat = sm.add_constant(X_mat)
model = sm.OLS(y_mat,X_mat)
results = model.fit()
print(results.params)

Processed Samples: 100%|████████████| 5097/5097 [00:00<00:00, 793820.04sample/s]

******************************
post-matching balance check reseult        Covariates  Mean Treated post-match  Mean Control post-match   SMD  \
0             c_1                   0.5052                   0.5145 -0.03   
1             c_2                   0.5026                   0.4954  0.03   
2             c_3                   0.5090                   0.5003  0.03   
3         d_1_bee                   0.1123                   0.1097  0.01   
4         d_1_cat                   0.1584                   0.1446  0.04   
5         d_1_dog                   0.1934                   0.1828  0.03   
6        d_1_pear                   0.2567                   0.2787 -0.05   
7     gender_cat1                   0.0721                   0.0708  0.01   
8     gender_cat2                   0.0865                   0.0861  0.00   
9     gender_cat3                   0.1717                   0.2007 -0.08   
10    gender_cat4                   0.0210                   0.0151  0.04   
11    gen

In [5]:
match_obj.sensitivity_test(gamma = [1,1.5,2,2.5,3])

AttributeError: 'DataFrame' object has no attribute '_append'

In [13]:
df_res_full.groupby(['y']).

y
y     1.5
y2    1.0
Name: gamma, dtype: float64

# debug

In [1]:
import sys
print(sys.path)
sys.path.append('/Users/bytedance/PycharmProjects/github/CausalMatch')

import causalmatch as causalmatch
from causalmatch import matching,gen_test_data

print('current version is: ',causalmatch.__version__)

import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)


import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,roc_auc_score,f1_score
import statsmodels.api as sm
from scipy.stats import wilcoxon,norm

df = pd.read_csv('/Users/bytedance/Downloads/sample_df.csv')
df.head()

['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/bytedance/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages']
current version is:  0.0.2
/usr/local/bin/python3.12
3.12.2 (v3.12.2:6abddd9f6a, Feb  6 2024, 17:02:06) [Clang 13.0.0 (clang-1300.0.29.30)]
sys.version_info(major=3, minor=12, micro=2, releaselevel='final', serial=0)


,live_watch_cnt_30,live_watch_interval_30,video_watch_cnt_30,video_watch_interval_30,prod_show_cnt_30,prod_click_cnt_30,query_moudle_click_cnt_30,search_click_cnt_30,icon_entrance_click_cnt_30,mall_search_mgr_search_dd_30,...,gmv_1d_aweme,gmv_30d,paydate_30d,电商大盘生命周期_平台潜客,电商大盘生命周期_成熟期中低频,电商大盘生命周期_成熟期沉默,电商大盘生命周期_成熟期高频,电商大盘生命周期_成长期沉默,电商大盘生命周期_成长期活跃,电商大盘生命周期_流失用户
0,0.0,0.0,0.0,0.0,157.0,9.0,8.0,1.0,0.0,0.0,...,467.39,1087.59,7,0,0,0,0,0,1,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.99,5.99,1,0,1,0,0,0,0,0
2,0.0,0.0,0.0,0.0,78.0,4.0,0.0,0.0,0.0,0.0,...,137.79,523.90,5,1,0,0,0,0,0,0
3,13.0,1157.0,15.0,28.0,490.0,16.0,13.0,2.0,0.0,2.0,...,143.40,603.90,5,0,1,0,0,0,0,0
4,0.0,0.0,5.0,27.0,302.0,6.0,60.0,0.0,0.0,2.0,...,110.09,239.57,7,0,0,0,1,0,0,0


In [2]:
df_test = df.sample(n=10000).copy()
df_test.reset_index(inplace=True, drop =True)

In [3]:
X = ['live_watch_cnt_30', 'live_watch_interval_30', 'video_watch_cnt_30',
       'video_watch_interval_30', 'prod_show_cnt_30', 'prod_click_cnt_30',
       'query_moudle_click_cnt_30', 'search_click_cnt_30',
       'icon_entrance_click_cnt_30', 'mall_search_mgr_search_dd_30',
       'search_mgr_manual_search_dd_30',
       'unmall_search_mgr_ecintent_search_cnt_1d',
       'mall_search_mgr_search_cnt_30d', 'manual_search_cnt_30d',
       'search_mgr_manual_search_cnt_30d', 'gmv_7d', 'gmv_14d', 'gmv_21d',
       'gmv_28d', 'order_cnt_7d', 'order_cnt_14d', 'order_cnt_21d',
       'order_cnt_28d', 'pay_order_cnt_14', 'pay_order_cnt_30', 'pay_amt_14',
       'pay_amt_30', 'gender', 'age', 'residence_city_level',
       'consumption_level_new', 'active_days', 'device_overall_score',
       'price_sensitive_prob_v2', 'follow_ecom_author', 'follow_live_author',
       'content_live_ecom_show_cnt', 'content_live_ecom_watch_cnt',
       'content_live_ecom_watch_duration', 'content_video_ecom_show_cnt',
       'content_video_ecom_watch_cnt', 'content_video_ecom_watch_duration',
       '电商大盘生命周期_平台潜客',
       '电商大盘生命周期_成熟期中低频', '电商大盘生命周期_成熟期沉默', '电商大盘生命周期_成熟期高频', '电商大盘生命周期_成长期沉默',
       '电商大盘生命周期_成长期活跃', '电商大盘生命周期_流失用户']
# y = ['gmv_1d_aweme','gmv_30d', 'paydate_30d']
y = ['gmv_30d']
id = 'user_id'
T = 'is_treatment'

In [4]:
# STEP 1: initialize matching object
match_obj = matching(data = df_test,
                     T = T,
                     X = X,
                     y = y,
                     id = id)

# STEP 2: propensity score matching
match_obj.psm(n_neighbors = 1,  # number of neighbors
              model = GradientBoostingClassifier(), # p-score model
              trim_percentage = 0.1, # trim X% of pscore, if equals 0.1 then trim min 5% and max 5%
              caliper = 0.1,
              verbose = True) # p-score diff must be smaller than or equal to the caliper value

# STEP 3: balance check after propensity score matching
res_post, res_pre = match_obj.balance_check(include_discrete = True)
print('post-matching balance check reseult',res_post) # pre-match balance check
print('pre-matching balance check reseult',res_pre)  # post-match balance check

# STEP 4: get result - pandas df, and merge X and y back to original data
print('Output dataframe columns', match_obj.df_out_final_post_trim.columns)
df_out = match_obj.df_out_final_post_trim.merge(df[y + X + [id]], how='left', on = id)

# STEP 5: calculate average treatment effect 

X_mat = df_out[T]
y_mat = df_out[y]

X_mat = sm.add_constant(X_mat)
model = sm.OLS(y_mat,X_mat)
results = model.fit()
print(results.params)

Processed Samples: 100%|███████████| 1628/1628 [00:00<00:00, 1099215.54sample/s]
/Users/bytedance/PycharmProjects/github/CausalMatch/causalmatch/matching/match_core/utils.py:179: RuntimeWarning: divide by zero encountered in scalar divide
  smd = np.round((t_avg - c_avg) / sf, 2)


******************************
post-matching balance check reseult                                   Covariates  Mean Treated post-match  \
0                          live_watch_cnt_30             3.117600e+00   
1                     live_watch_interval_30             1.472367e+02   
2                         video_watch_cnt_30             5.895000e+00   
3                    video_watch_interval_30             2.889290e+01   
4                           prod_show_cnt_30             2.557318e+02   
5                          prod_click_cnt_30             8.943300e+00   
6                  query_moudle_click_cnt_30             4.481100e+00   
7                        search_click_cnt_30             6.022000e-01   
8                 icon_entrance_click_cnt_30             0.000000e+00   
9               mall_search_mgr_search_dd_30             6.099000e-01   
10            search_mgr_manual_search_dd_30             2.000000e+00   
11  unmall_search_mgr_ecintent_search_cnt_1d             

const           175.040712
is_treatment    430.432993
dtype: float64


In [5]:
match_obj.sensitivity_test(gamma = [1,1.5,2,2.5,3])

,Wilcoxon-statistic,gamma,stat upper bound,stat_lower_bound,z-score upper bound,z-score lower bound,p-val upper bound,p-val lower bound,y
0,836077.5,1.0,600127.50,600127.50,12.623085,12.623085,0.0000,0.0,gmv_30d
1,836077.5,1.5,720153.00,480102.00,6.329729,19.437035,0.0000,0.0,gmv_30d
2,836077.5,2.0,800170.00,400085.00,2.037544,24.740064,0.0208,0.0,gmv_30d
3,836077.5,2.5,857325.00,342930.00,NaN,NaN,NaN,NaN,gmv_30d
4,836077.5,3.0,900191.25,300063.75,NaN,NaN,NaN,NaN,gmv_30d


In [6]:
y_i = 'gmv_30d'

In [7]:
id_list_t = match_obj.df_out_final['user_id_treat'].to_list()
id_list_c = match_obj.df_out_final['user_id_control'].to_list()

df_pair = pd.DataFrame()
# ------------------------------------------------------------------------------
# BUGFIX: 20241018
# df_pair['y_t'], df_pair['y_c'] = match_obj.data.iloc[id_list_t][y_i].values, match_obj.data.iloc[id_list_c][y_i].values
df_pair['y_t'] = match_obj.data[match_obj.data[match_obj.id].isin(id_list_t)][y_i].values
df_pair['user_id_treat'] = id_list_t
df_pair['user_id_control'] = id_list_c
df_pair_ = df_pair.merge(match_obj.data[[match_obj.id, y_i]], how='left', left_on='user_id_control',
                         right_on=match_obj.id)
df_pair_.rename(columns={y_i : "y_c"}, inplace=True)
df_pair_.drop(columns=match_obj.id, inplace=True)
df_pair = df_pair_.copy()
# ------------------------------------------------------------------------------

df_pair['absolute_diff'] = np.abs(df_pair['y_t'] - df_pair['y_c'])
df_pair['absolute_diff_rank'] = df_pair['absolute_diff'].rank()
df_pair['cs1'] = (df_pair['y_t'] > df_pair['y_c']) * 1
df_pair['cs2'] = (df_pair['y_c'] > df_pair['y_t']) * 1
df_pair['cz'] = df_pair['cs1'] * 1 + df_pair['cs2'] * 0
df_pair['dcz'] = df_pair['cz'] * df_pair['absolute_diff_rank']
test_stat = df_pair['dcz'].sum()

cols = ['Wilcoxon-statistic', 'gamma'
        , 'stat upper bound', 'stat_lower_bound'
        , 'z-score upper bound', 'z-score lower bound'
        , 'p-val upper bound', 'p-val lower bound']
lst = []

In [9]:
gamma_i = 2.5

In [10]:
df_pair['ps_plus'] = gamma_i / (1 + gamma_i)
df_pair['ps_plus'][(df_pair['cs1'] == 0) & (df_pair['cs2'] == 0)] = 0
df_pair['ps_plus'][(df_pair['cs1'] == 1) & (df_pair['cs2'] == 1)] = 1

df_pair['ps_minus'] = 1 / (1 + gamma_i)
df_pair['ps_minus'][(df_pair['cs1'] == 0) & (df_pair['cs2'] == 0)] = 0
df_pair['ps_minus'][(df_pair['cs1'] == 1) & (df_pair['cs2'] == 1)] = 1

gamma_1_t_plus = np.sum(df_pair['absolute_diff_rank'] * df_pair['ps_plus'])

gamma_1_t_minus = np.sum(df_pair['absolute_diff_rank'] * df_pair['ps_minus'])

var_t_plus = np.sum(
    df_pair['absolute_diff_rank'] ** 2 * (df_pair['ps_plus'] * (1 - df_pair['ps_plus'])))

deviate_ub = (df_pair['dcz'].sum() - gamma_1_t_plus) / np.sqrt(var_t_plus)
deviate_lb = (df_pair['dcz'].sum() - gamma_1_t_minus) / np.sqrt(var_t_plus)

In [11]:
deviate_ub

-1.2581181067666987

In [12]:
deviate_lb

29.200508250699166

In [13]:
round(gamma_1_t_plus, 2)

857325.0

In [58]:
df_pair = pd.DataFrame()
# df_pair['y_t'] = match_obj.data.iloc[id_list_t][y_i].values
# df_pair['y_c'] = match_obj.data.iloc[id_list_c][y_i].values

df_pair['y_t'] = match_obj.data[match_obj.data[match_obj.id].isin(id_list_t)][y_i].values
# df_pair['y_c'] = match_obj.data[match_obj.data[match_obj.id].isin(id_list_c)][y_i].values
df_pair['user_id_treat'] = id_list_t
df_pair['user_id_control'] = id_list_c

df_pair_ = df_pair.merge(match_obj.data[[match_obj.id,y_i]], how='left', left_on='user_id_control', right_on=match_obj.id)
df_pair_.rename(columns={y_i: "y_c"}, inplace=True)
df_pair_.drop(columns=match_obj.id, inplace=True)
df_pair = df_pair_.copy()


In [4]:
# df_out = match_obj.df_out_final_post_trim.merge(df[y + X + [id]], how='left', on = id)

In [5]:
df_out_final = match_obj.df_out_final.copy()

y_i = y[0] # Feature 1: change to loop later

# for y_i in y:

id_list_t = list(df_out_final['user_id_treat'].values)
id_list_c = list(df_out_final['user_id_control'].values)

y_t = df.iloc[id_list_t][y_i].values
y_c = df.iloc[id_list_c][y_i].values

df_out_final['y_t'] = y_t
df_out_final['y_c'] = y_c


In [16]:
# df_out_final['absolute_diff'] = np.round(np.abs(df_out_final['y_t'] - df_out_final['y_c']),2)
df_out_final['absolute_diff'] = np.abs(df_out_final['y_t'] - df_out_final['y_c'])
df_out_final['absolute_diff_rank'] = df_out_final['absolute_diff'].rank()
df_out_final['cs1'] = (df_out_final['y_t']>df_out_final['y_c']) * 1
df_out_final['cs2'] = (df_out_final['y_c']>df_out_final['y_t']) * 1
df_out_final['cz']  = df_out_final['cs1'] * 1 + df_out_final['cs2'] * 0

df_out_final['dcz'] = df_out_final['cz'] * df_out_final['absolute_diff_rank']
print(df_out_final['dcz'].sum())
print(wilcoxon(y_t, y_c, alternative='greater'))



7814466.0
WilcoxonResult(statistic=7814466.0, pvalue=2.032795203519004e-36)


In [10]:
gamma_list_array =  np.array(list(range(0,100,10)))/20
gamma_list = gamma_list_array[gamma_list_array>=1]
gamma_list

array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5])

In [14]:
cols = ['gamma','e_t_plus','e_t_minus','z_plus','z_minus','p_plus','p_minus']

lst = []
for gamma_i in gamma_list:

    df_out_final['ps_plus'] = gamma_i/(1 + gamma_i)
    df_out_final['ps_plus'][(df_out_final['cs1']==0) & (df_out_final['cs2']==0)] = 0
    df_out_final['ps_plus'][(df_out_final['cs1']==1) & (df_out_final['cs2']==1)] = 1

    df_out_final['ps_minus'] = 1/(1 + gamma_i)
    df_out_final['ps_minus'][(df_out_final['cs1']==0) & (df_out_final['cs2']==0)] = 0
    df_out_final['ps_minus'][(df_out_final['cs1']==1) & (df_out_final['cs2']==1)] = 1

    gamma_1_t_plus = np.sum(df_out_final['absolute_diff_rank'] * df_out_final['ps_plus'])

    gamma_1_t_minus = np.sum(df_out_final['absolute_diff_rank'] * df_out_final['ps_minus'])


    var_t_plus = np.sum(df_out_final['absolute_diff_rank'] ** 2 * (df_out_final['ps_plus'] * (1-df_out_final['ps_plus'])))

    deviate_ub = (df_out_final['dcz'].sum() - gamma_1_t_plus) / np.sqrt(var_t_plus)
    deviate_lb = (df_out_final['dcz'].sum() - gamma_1_t_minus) / np.sqrt(var_t_plus)
    
    lst.append([gamma_i
          , round(gamma_1_t_plus,2)
          , round(gamma_1_t_minus,2)
          , deviate_ub
          , deviate_lb
          , np.round(norm.sf(abs(deviate_ub)),2)
          , np.round(norm.sf(abs(deviate_lb)),2)])

df_sensitivity_test = pd.DataFrame(lst, columns=cols)
df_sensitivity_test

,gamma,e_t_plus,e_t_minus,z_plus,z_minus,p_plus,p_minus
0,1.0,6496126.50,6496126.50,12.548218,12.548218,0.00,0.0
1,1.5,7795351.80,5196901.20,0.185684,25.428257,0.43,0.0
2,2.0,8661502.00,4330751.00,-8.551315,35.170104,0.00,0.0
3,2.5,9280180.71,3712072.29,-15.440888,43.217551,0.00,0.0
4,3.0,9744189.75,3248063.25,-21.208956,50.187823,0.00,0.0
5,3.5,10105085.67,2887167.33,-26.221423,56.404293,0.00,0.0
6,4.0,10393802.40,2598450.60,-30.688296,62.058840,0.00,0.0
7,4.5,10630025.18,2362227.82,-34.741284,67.275360,0.00,0.0


In [27]:
df_out_final = pd.DataFrame()
df_out_final['y_t'] = [16,14,22,21,10,13,20,23,18,24,17,15,34,23,25,27,31,34,35,39,37,38,41,35,36,39,44,48,43,45,49,62,73]
df_out_final['y_c'] = [16,13,21,19,13,16,16,18,24,18,10,24,25,10,11,13,16,18,19,22,19,16,18,12,11,14,19,18,11,9,7,15,13]

# df_out_final['absolute_diff'] = np.round(np.abs(df_out_final['y_t'] - df_out_final['y_c']),2)
df_out_final['absolute_diff'] = np.abs(df_out_final['y_t'] - df_out_final['y_c'])
df_out_final['absolute_diff_rank'] = df_out_final['absolute_diff'].rank()
df_out_final['cs1'] = (df_out_final['y_t']>df_out_final['y_c']) * 1
df_out_final['cs2'] = (df_out_final['y_c']>df_out_final['y_t']) * 1
df_out_final['cz']  = df_out_final['cs1'] * 1 + df_out_final['cs2'] * 0

df_out_final['dcz'] = df_out_final['cz'] * df_out_final['absolute_diff_rank']
print(df_out_final['dcz'].sum())
# print(wilcoxon(df_out_final['y_t'].values, df_out_final['y_c'].values))
print(wilcoxon(df_out_final['y_t'].values, df_out_final['y_c'].values, alternative='greater'))


gamma_list_array =  np.array(list(range(0,100,10)))/20
gamma_list = gamma_list_array[gamma_list_array>=1]

cols = ['gamma','e_t_plus','e_t_minus','z_plus','z_minus','p_plus','p_minus']

lst = []
for gamma_i in gamma_list:

    df_out_final['ps_plus'] = gamma_i/(1 + gamma_i)
    df_out_final['ps_plus'][(df_out_final['cs1']==0) & (df_out_final['cs2']==0)] = 0
    df_out_final['ps_plus'][(df_out_final['cs1']==1) & (df_out_final['cs2']==1)] = 1

    df_out_final['ps_minus'] = 1/(1 + gamma_i)
    df_out_final['ps_minus'][(df_out_final['cs1']==0) & (df_out_final['cs2']==0)] = 0
    df_out_final['ps_minus'][(df_out_final['cs1']==1) & (df_out_final['cs2']==1)] = 1

    gamma_1_t_plus = np.sum(df_out_final['absolute_diff_rank'] * df_out_final['ps_plus'])

    gamma_1_t_minus = np.sum(df_out_final['absolute_diff_rank'] * df_out_final['ps_minus'])


    var_t_plus = np.sum(df_out_final['absolute_diff_rank'] ** 2 * (df_out_final['ps_plus'] * (1-df_out_final['ps_plus'])))

    deviate_ub = (df_out_final['dcz'].sum() - gamma_1_t_plus) / np.sqrt(var_t_plus)
    deviate_lb = (df_out_final['dcz'].sum() - gamma_1_t_minus) / np.sqrt(var_t_plus)
    
    lst.append([gamma_i
          , round(gamma_1_t_plus,2)
          , round(gamma_1_t_minus,2)
          , deviate_ub
          , deviate_lb
          , np.round(norm.sf(abs(deviate_ub)),5)
          , np.round(norm.sf(abs(deviate_lb)),5)])

df_sensitivity_test = pd.DataFrame(lst, columns=cols)
df_sensitivity_test

527.0
WilcoxonResult(statistic=499.0, pvalue=5.5308015357736255e-06)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  res = hypotest_fun_out(*samples, **kwds)


,gamma,e_t_plus,e_t_minus,z_plus,z_minus,p_plus,p_minus
0,1.0,280.00,280.00,4.414499,4.414499,0.00001,0.00001
1,1.5,336.00,224.00,3.484033,5.527026,0.00025,0.00000
2,2.0,373.33,186.67,2.912999,6.451567,0.00179,0.00000
3,2.5,400.00,160.00,2.512212,7.259698,0.00600,0.00000
4,3.0,420.00,140.00,2.208196,7.986653,0.01362,0.00000
5,3.5,435.56,124.44,1.965578,8.652842,0.02467,0.00000
6,4.0,448.00,112.00,1.764906,9.271342,0.03879,0.00000
7,4.5,458.18,101.82,1.594462,9.851120,0.05542,0.00000


# 20240929: add refute test
refute test types refers to https://causalwizard.app/inference/article/bootstrap-refuters-dowhy

In [1]:
import sys
print(sys.path)
sys.path.append('/Users/bytedance/PycharmProjects/github/CausalMatch')

import causalmatch as causalmatch
from causalmatch import matching,gen_test_data

print('current version is: ',causalmatch.__version__)

import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)


import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,roc_auc_score,f1_score
import statsmodels.api as sm
from scipy.stats import wilcoxon,norm
import copy

df, rand_continuous, rand_true_param, param_te , rand_treatment, rand_error = gen_test_data(n = 10000, c_ratio=0.5)
X = ['c_1', 'c_2', 'c_3', 'd_1', 'gender']
# y = ['y', 'y2']
y = ['y']
id = 'user_id'
T = 'treatment'


match_obj = matching(data = df, T = T, X = X, y = y, id = id)

match_obj.psm(n_neighbors = 1, 
              model = LogisticRegression(), 
              trim_percentage = 0.1, 
              caliper = 0.1,
              verbose = False) 

true_ate = match_obj.ate().iloc[0]['ate']


['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/bytedance/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages']
current version is:  0.0.2
/usr/local/bin/python3.12
3.12.2 (v3.12.2:6abddd9f6a, Feb  6 2024, 17:02:06) [Clang 13.0.0 (clang-1300.0.29.30)]
sys.version_info(major=3, minor=12, micro=2, releaselevel='final', serial=0)


Processed Samples: 100%|████████████| 5097/5097 [00:00<00:00, 827688.55sample/s]

******************************


In [2]:
gamma = [1,1.5,2,2.5,3]
match_obj.robust_check(gamma)

             Robustness Check Output Table for Dep. Variable y              
                                                            coef    P>|t|   
  Average Treatment Effect:                                 0.51     0.00%  
                                                                            
  1. Total % of obs remained:                             91.76%      Pass  
   -- Treated % obs remained:                             90.05%      Pass  
   -- Control % obs remained:                             93.53%      Pass  
      -- The most repeated times of a control obs:             9         -  
  2. Sensitivity test result, Gamma statistics               1.0      Fail  
  3. Placebo Test Result conf. interval              -0.33,0.33,      Pass  


In [3]:
# summary_stat = {"y" : self.y[0],
#                 "ATE": '{:.2f}'.format(self.ate().iloc[0]['ate']),
#                 "remain_sample_fraction": '{:.2f}%'.format(remain_sample_fraction*100),
#                 "remain_sample_fraction_t": '{:.2f}%'.format(remain_sample_fraction_t*100),
#                 "remain_sample_fraction_c": '{:.2f}%'.format(remain_sample_fraction_c*100),
#                 "max_repeat_time": max_repeat_time,
#                 "max_repeat_control_id_list_str": max_repeat_control_id_list_str,
#                 "gamma_test_stat": gamma_test_stat,
#                 "conf. interval": critical_val_str}

In [4]:
from statsmodels.compat.python import lrange, lmap, lzip
from statsmodels.iolib.table import SimpleTable
from itertools import zip_longest

In [11]:
match_obj.ate()

,y,ate,p_val
0,y,0.509743,3.503957e-26


'0.00%'

In [28]:


# Simpletable should be able to handle the formating
params_data_ = (['{:.2f}'.format(match_obj.ate().iloc[0]['ate']), '{:.2f}%'.format(match_obj.ate().p_val[0])],
                ['{:.2f}%'.format(remain_sample_fraction*100),5],
                ['{:.2f}%'.format(0.99*100),5],
                ['{:.2f}%'.format(0.99*100),5],
                [9,5],
                ['asasda',5],
                [1,5],                    
                ['0,0',8])

params_stubs_ = ['Average Treatment Effect: '
                 ,'Total obs remain fraction'
                 ,'Treated obs remain fraction'
                 ,'Control obs remain fraction'
                 ,'The maximun repeated times of a control group observation'
                 ,'The list of control group id that repeat the most'
                 ,'Sensitivity test result, Gamma statistics'
                 ,'Sensitivity test result, Gamma statistics'
                 ,'Placebo Test Result conf. interval']



gen_fmt = {
    "data_fmts": ["%s", "%s", "%s", "%s", "%s"],
    "empty_cell": '',
    "colwidths": 7,
    "colsep": '   ',
    "row_pre": '  ',
    "row_post": '  ',
    "table_dec_above": '=',
    "table_dec_below": None,
    "header_dec_below": None,
    "header_fmt": '%s',
    "stub_fmt": '%s',
    "title_align": 'c',
    "header_align": 'r',
    "data_aligns": "r",
    "stubs_align": "l",
    "fmt": 'txt'
}

gen_title = 'Robustness Check Output Table for Dep. Variable {}'.format(match_obj.y[0])
gen_header = ['coef','P>|t| ']


gen_table_parm = SimpleTable(params_data_,
                             gen_header,
                             params_stubs_,
                             title=gen_title,
                             txt_fmt=gen_fmt
                             )


In [29]:
print(gen_table_parm)
# print(gen_table_left)
# print(gen_table_right)

                Robustness Check Output Table for Dep. Variable y                
                                                                 coef    P>|t|   
  Average Treatment Effect:                                      0.51     0.00%  
  Total obs remain fraction                                    99.00%         5  
  Treated obs remain fraction                                  99.00%         5  
  Control obs remain fraction                                  99.00%         5  
  The maximun repeated times of a control group observation         9         5  
  The list of control group id that repeat the most            asasda         5  
  Sensitivity test result, Gamma statistics                         1         5  
  Sensitivity test result, Gamma statistics                       0,0         8  


In [23]:
# print(gen_table_left)
# print(gen_table_right)

coef nonrobust std err          t      P>|t|     

In [28]:
gen_stubs_right

('pass', 'fail', 'pass', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')

In [29]:
gen_data_middle

('a', 'b', 'c', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')

In [30]:
gen_data_right

('1', '2', '3', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')

In [2]:
from causalmatch.matching.match_core.utils import data_process_bc
df_out_ = match_obj.df_out_final

# 多少observation的数量被drop了
match_obj.df_out_final_post_trim

# control样本最大使用次数：哪一个control的obs被用的最多；
X_balance_check, df_post_validate, df_pre_validate = data_process_bc(match_obj, True)


In [3]:
# 总体remain了多少样本
remain_sample_fraction = df_post_validate.shape[0]/df_pre_validate.shape[0]

# 实验组remain了多少样本
remain_sample_fraction_t = df_post_validate[df_post_validate[match_obj.T]==1].shape[0]/df_pre_validate[df_pre_validate[match_obj.T]==1].shape[0]

# 对照组remain了多少样本
remain_sample_fraction_c = df_post_validate[df_post_validate[match_obj.T]!=1].shape[0]/df_pre_validate[df_pre_validate[match_obj.T]!=1].shape[0]

print(remain_sample_fraction, remain_sample_fraction_t, remain_sample_fraction_c)


0.9176 0.9005297233666862 0.9353457067101775


In [4]:
# match_obj.T
# match_obj.id


In [5]:
df_repeat_ = pd.DataFrame(df_post_validate[df_post_validate[match_obj.T]!=1][match_obj.id].value_counts())
df_repeat_.reset_index(inplace=True)
max_repeat_time = df_repeat_[match_obj.id].max()


max_repeat_control_id_list = list(df_repeat_[df_repeat_[match_obj.id]==max_repeat_time]['index'].values)

max_repeat_control_id_list_str = ''.join(str(e)+',' for e in max_repeat_control_id_list)
max_repeat_control_id_list_str

'5785,3981,3793,7516,'

In [11]:
from causalmatch.matching.match_core.robust_check import sensitivity_test

gamma = [1,1.5,2,2.5,3]

df_res_full = pd.DataFrame()
for y_i in match_obj.y:
    df_res = sensitivity_test(match_obj, gamma, y_i)
    df_res['y'] = y_i
    df_res_full = pd.concat([df_res_full, df_res], ignore_index=True)

df_gamma_ = df_res_full[~df_res_full['z-score upper bound'].isna()].groupby(['y']).max()['gamma']
df_gamma_.values[0]

1.0

'psm'

In [7]:
col_list = ['y',
            'remain_sample_fraction',
            'remain_sample_fraction_t',
            'remain_sample_fraction_c',
            'max_repeat_time',
            'max_repeat_time',]

summary_stat = {"y" : match_obj.y[0],
               "remain_sample_fraction": remain_sample_fraction,
               "remain_sample_fraction_t": remain_sample_fraction_t, 
               "remain_sample_fraction_c": remain_sample_fraction_c,}

summary_stat

{'y': 'y',
 'remain_sample_fraction': 0.9176,
 'remain_sample_fraction_t': 0.9005297233666862,
 'remain_sample_fraction_c': 0.9353457067101775}

In [8]:
# Simpletable should be able to handle the formating
df_gamma_

NameError: name 'df_gamma_' is not defined

In [24]:
?pd.DataFrame

In [3]:
# 1. 多少observation的数量被drop了
# 2. control样本最大使用次数：哪一个control的obs被用的最多；
# 3. 敏感性分析，gamma检验结果
# 4. 反驳式：
#    4.1 placebo
#    4.2 添加随机混淆检验
#    4.3 子集检验

In [2]:
n = 1000
b = 100
np.random.seed(123456)

# Sample sub data from original dataset
data_b = match_obj.data.sample(n=n).copy()
data_b.reset_index(inplace=True, drop=True)

lb_index = np.round(b * 0.025)
ub_index = np.round(b * (1 - 0.025))

# Create random treatment variable based on binomial(0.5) distribution
rand_discrete = np.random.binomial(1, 0.5, [n, b])
ate_list = []

pseudo_t_list = []


# match_obj = matching(data = df, T = t_i_name, X = X, y = ['y'], id = id)
    
    
for i in range(b):
    print(i)
    t_i_name = 't_{}'.format(i)
    pseudo_t_list.append(t_i_name)
    data_b[t_i_name] = rand_discrete[:, i]

#     match_obj_i = matching(data = data_b   # This parm change with loop
#                            , T = t_i_name  # This parm change with loop
#                            , X = match_obj.X
#                            , y = match_obj.y
#                            , id = match_obj.id
#                            , method = match_obj.method)

    match_obj_i = copy.deepcopy(match_obj)
    match_obj_i.data = data_b
    match_obj_i.T = t_i_name
    match_obj_i.preprocess()
    
    match_obj_i.psm(n_neighbors     = 1,
                    model           = match_obj.model,
                    caliper         = match_obj.caliper,
                    trim_percentage = match_obj.trim_percentage,
                    drop_duplicates = match_obj.drop_duplicates,
                    model_list      = match_obj.model_list,
                    test_size       = match_obj.test_size,
                    verbose         = None)
    
    ate_i = match_obj_i.ate().iloc[0]['ate']
    ate_list.append(ate_i)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


/var/folders/6y/pv_cpv891fb7v3w2vj8fsz8m0000gn/T/ipykernel_88507/1245409651.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_b[t_i_name] = rand_discrete[:, i]
/var/folders/6y/pv_cpv891fb7v3w2vj8fsz8m0000gn/T/ipykernel_88507/1245409651.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_b[t_i_name] = rand_discrete[:, i]
/var/folders/6y/pv_cpv891fb7v3w2vj8fsz8m0000gn/T/ipykernel_88507/1245409651.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many time

In [1]:
ate_list

NameError: name 'ate_list' is not defined

In [1]:
import sys
print(sys.path)
sys.path.append('/Users/bytedance/PycharmProjects/github/CausalMatch')

import causalmatch as causalmatch
from causalmatch import matching,gen_test_data

print('current version is: ',causalmatch.__version__)

import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)


import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,roc_auc_score,f1_score
import statsmodels.api as sm
from scipy.stats import wilcoxon,norm

df = pd.read_csv('/Users/bytedance/Downloads/dual_stable_matching_sample_data_df_v2.csv')

df.head()

['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/bytedance/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages']
current version is:  0.0.2
/usr/local/bin/python3.12
3.12.2 (v3.12.2:6abddd9f6a, Feb  6 2024, 17:02:06) [Clang 13.0.0 (clang-1300.0.29.30)]
sys.version_info(major=3, minor=12, micro=2, releaselevel='final', serial=0)


,device_id,version_id,if_stable_show,user_id,min_date,recommend_age_range,recommend_gender,manual_search_activeness,if_xhs_user,is_shuangchi_xhs,...,after_search_dcol_enter_cnt,after_search_dcol_show_cnt,after_search_dcol_click_cnt,after_search_dcol_vv_1d,after_search_dcol_play_duration,after_enter_homepage_mediumvideo_cnt,after_mediumvideo_client_show_cnt,after_mediumvideo_client_click_cnt,after_mediumvideo_vv,after_mediumvideo_play_duration
0,68203691699,9965363,0,1.003663e+11,20240826.0,41-50,男,5手搜新用户,1.0,0.0,...,5.0,61.0,5.0,5.0,31.171,0.0,0.0,0.0,0.0,0.0
1,70948228012,9965363,0,3.861097e+15,20241006.0,41-50,男,5手搜新用户,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0
2,4191768220541288,9965363,0,5.273608e+14,20240928.0,50+,男,5手搜新用户,1.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0
3,257699537496412,9965363,0,3.760758e+15,20240920.0,41-50,男,5手搜新用户,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0
4,541377775799469,9965363,0,7.300281e+10,20240827.0,18-23,男,2手搜中高活,0.0,0.0,...,1.0,5.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0


## 1. placebo treatment 

In [2]:

# -- Verifies the effect disappears when the Treatment is replaced with a placebo.
# -- generate 100 random treatment variables

np.random.seed(123456)

b = 100 
lb_index = np.round(b* 0.025)
ub_index = np.round(b* (1-0.025))

n_obs = df.shape[0]
rand_discrete = np.random.binomial(1,0.5,[n_obs, b])
rand_discrete
ate_list = []

pseudo_t_list = []

for i in range(b):    
    t_i_name = 't_{}'.format(i)
    pseudo_t_list.append(t_i_name)    
    df[t_i_name] = rand_discrete[:,i]
    
    match_obj = matching(data = df, T = t_i_name, X = X, y = ['y'], id = id)
    match_obj.psm(n_neighbors = 1, 
                  model = LogisticRegression(), 
                  trim_percentage = 0.1, 
                  caliper = 0.1,
                  verbose = False) 

    df_ate = match_obj.ate()
    ate_i = df_ate.iloc[0]['ate']
    ate_list.append(ate_i)
    


Processed Samples: 100%|████████████| 4919/4919 [00:00<00:00, 295647.80sample/s]

******************************



Processed Samples: 100%|████████████| 4972/4972 [00:00<00:00, 475198.35sample/s]


******************************


Processed Samples: 100%|████████████| 5044/5044 [00:00<00:00, 362044.48sample/s]


...

******************************


/var/folders/6y/pv_cpv891fb7v3w2vj8fsz8m0000gn/T/ipykernel_77091/89093489.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[t_i_name] = rand_discrete[:,i]
Processed Samples: 100%|████████████| 5001/5001 [00:00<00:00, 464526.95sample/s]
/var/folders/6y/pv_cpv891fb7v3w2vj8fsz8m0000gn/T/ipykernel_77091/89093489.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[t_i_name] = rand_discrete[:,i]


******************************


Processed Samples: 100%|████████████| 4930/4930 [00:00<00:00, 403408.61sample/s]

******************************


In [3]:
critical_value_lb = np.sort(np.array(ate_list))[int(lb_index)]
critical_value_ub = np.sort(np.array(ate_list))[int(ub_index)]

if (true_ate<=critical_value_ub) and (true_ate>=critical_value_lb):
    print('fail placebo test')
else:
    print('pass placebo test')

pass placebo test


In [10]:
import copy

match_obj.data

## 2. add unobserved common cause

add an additional common cause to the dataset that is correlated with the treatment and the outcome.

## 3. data subsets validation